# 5. Fourier Transform of 3D Images
* Notebooks hereafter have been inspired by [DENSS - Thomas D. Grant](https://www.tdgrant.com/denss/) and use a small portion of its source code. 
* For further study, see also this author's paper [Ab initio electron density determination directly from solution scattering data](https://www.nature.com/articles/nmeth.4581)
* First, we will be observing just the results of Fourier Transform.
* More difficult part, spherical average in the reciprocal space, will be discussed in later notebooks.

## 5.1 From Real Space to Reciporcal Space
* For this transform, use np.fft.fftn().

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib widget

In [2]:
N = 32
x = y = z = np.arange(N)
xx, yy, zz = np.meshgrid(x, y, z)

In [3]:
def plot_a_sphere_with_ft(center, r):
    import matplotlib.cm as cm
    from learnsaxs import draw_voxles_as_dots
    cx, cy, cz = center
    shape = (xx - cx)**2 + (yy - cy)**2 + (zz - cz)**2 < r**2
    canvas = np.zeros((N,N,N))
    canvas[shape] = 1
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(8,4), subplot_kw={"projection":"3d"})
    ax1.set_title("Real Space Image")
    ax2.set_title("Reciprocal Space Image abs(F)")
    draw_voxles_as_dots(ax1, canvas)
    ft_image = np.abs(np.fft.fftn(canvas))
    draw_voxles_as_dots(ax2, ft_image)
    ax1.set_xlim(ax2.get_xlim())
    ax1.set_ylim(ax2.get_ylim())
    ax1.set_zlim(ax2.get_zlim())

In [4]:
plot_a_sphere_with_ft((16, 16, 16), 4)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 5.2 From Reciprocal Space to Real Space
* To invert the transform, use np.fft.ifftn(F).
* Note that we used F, not abs(F), to restore the original shape to avoid "[Phase Problem](https://en.wikipedia.org/wiki/Phase_problem)", which will be discussed in later notebooks.

In [5]:
def plot_a_sphere_with_ft_and_ift(center, r):
    import matplotlib.cm as cm
    from learnsaxs import draw_voxles_as_dots
    cx, cy, cz = center
    shape = (xx - cx)**2 + (yy - cy)**2 + (zz - cz)**2 < r**2
    canvas = np.zeros((N,N,N))
    canvas[shape] = 1
    fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, figsize=(12,4), subplot_kw={"projection":"3d"})
    ax1.set_title("Real Space Image")
    ax2.set_title("Reciprocal Space Image abs(F)")
    ax3.set_title("Restored Real Space Image")
    draw_voxles_as_dots(ax1, canvas)
    F = np.fft.fftn(canvas)
    ft_image = np.abs(F)
    draw_voxles_as_dots(ax2, ft_image)
    rest_image = np.fft.ifftn(F).real
    rest_image_ = np.round(rest_image, decimals=7)    # avoid slight differences to affect the color map
    draw_voxles_as_dots(ax3, rest_image_)
    for ax in [ax1, ax3]:
        ax.set_xlim(ax2.get_xlim())
        ax.set_ylim(ax2.get_ylim())
        ax.set_zlim(ax2.get_zlim())

In [6]:
plot_a_sphere_with_ft_and_ift((16, 16, 16), 4)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 5.3 Subtle but Unignorable Precision Problem
* In the above example, we used np.round() to avoid inconvenience.
* The problem seems to be related to the default color map calculation in matplotlib, on which we are not going into details hare.
* It is subtle, but confusing for demo purposes.

In [7]:
def debug_plot_a_sphere_with_ft_and_ift(center, r):
    cx, cy, cz = center
    shape = (xx - cx)**2 + (yy - cy)**2 + (zz - cz)**2 < r**2
    canvas = np.zeros((N,N,N))
    canvas[shape] = 1
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(8,4))
    F = np.fft.fftn(canvas)
    ft_image = np.abs(F)
    ret_image = np.fft.ifftn(F).real
    print("max(ret_image)=",  np.max(ret_image))
    print("min(ret_image)=",  np.min(ret_image[ret_image > 0]))    
    ax1.hist(canvas.flatten())
    ax2.hist(ret_image.flatten())

In [8]:
debug_plot_a_sphere_with_ft_and_ift((16, 16, 16), 4)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

max(ret_image)= 1.0
min(ret_image)= 1.4465731197425178e-21
